In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import pandas_profiling
%matplotlib inline

In [101]:
train_ = pd.read_csv('./data/train.csv',index_col=0)
test_ = pd. read_csv('./data/test.csv',index_col=0)
sample_submission = pd.read_csv('./data/sample_submission.csv', index_col=0)

데이터 설명

-     대전지역에서 측정한 실내외 19곳의 센서데이터와, 주변 지역의 기상청 공공데이터를 semi-비식별화하여 제공합니다. 
-     센서는 온도를 측정하였습니다. 
-     모든 데이터는 시간 순으로 정렬 되어 있으며 10분 단위 데이터 입니다. 
-     예측 대상(target variable)은 Y18입니다. 

train.csv 

-     30일 간의 기상청 데이터 (X00 ~ X39) 및 센서데이터 (Y00~Y17)
-     이후 3일 간의 기상청 데이터 (X00~X39) 및 센서데이터 (Y18)

test.csv 
-     train.csv 기간 이후 80일 간의 기상청 데이터 (X00~X39)

sample_submission.csv
-     제출 양식 예시

X00 X07 X28 X31 기온  
X01 X06 X22 X27 기압  
X02 X03 X18 X24 풍속  
X04 X10 X21 X36 누적강수량  
X05 X08 X09 X23 해면기압  
X11 X14 X16 X19 누적일사량  
X12, X20, X30, X37 습도
X13 X15 X17 X25 풍향  

Y00~17 센서측정온도  

In [102]:
train=train_.copy()
test=test_.copy()

### Y00 ~ Y17 >>> [0:4320]: Value, [4321:]: NaN, Y_index: [41:58]
### Y18 >>> [0:4320]: NaN, [4321:]: Value, Y_index: [59]

In [103]:
tp=0
k=0

In [104]:
for i in range(40,58):
    if i != 46 | i != 47 | i != 52:
        tp += train.iloc[0:4320,i]
        k+=1

In [6]:
train.shape

(4752, 59)

In [7]:
train.iloc[0:4320,58]=tp/k

In [8]:
train.head()

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,...,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
id,,,,,,,,,,,,,,,,,,,,,
0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,1009.8,...,7.0,7.5,7.0,9.0,10.0,9.5,9.0,8.0,9.0,10.9375
1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,1009.9,...,6.5,7.5,7.0,8.5,10.0,9.5,9.0,7.5,9.0,10.7500
2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,1010.1,...,6.5,7.5,6.5,8.0,9.5,9.5,8.5,7.5,8.5,10.5000
3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,1010.0,...,6.0,7.0,6.0,8.0,9.5,9.0,8.5,7.5,8.5,10.3750
4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,1010.0,...,6.0,7.0,6.0,7.5,9.5,9.0,8.5,7.5,8.5,10.1875


In [46]:
x=train.iloc[:,1:40]
y=train.iloc[:,-1]

In [47]:
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit

x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.8, shuffle=False) #random_state=0)


In [48]:
x_train.to_csv('x_train.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')

C:\Users\jungg\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
C:\Users\jungg\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [56]:
test_x=test.iloc[:,1:40]

In [88]:
from sklearn.preprocessing import StandardScaler, RobustScaler

scaler2=StandardScaler()

scaler2.fit(x_train)
x_train=scaler2.transform(x_train)
x_test=scaler2.transform(x_test)

#test_x=scaler2.transform(test_x)

# scaler1=StandardScaler()
# scaler1.fit(y_train)
# y_train=scaler1.transform(y_train)
# y_test=scaler1.transform(y_test)


In [89]:
# !pip install catboost

In [90]:
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgbm
import xgboost as xgb
import catboost as cb

from sklearn.metrics import r2_score, mean_squared_error

params={ 'max_depth':9, 'n_estimators':3500,  'learning_rate': 0.01, \
        'boost_from_average': False, 'subsample': 1}

clf = cb.CatBoostRegressor(**params)
model=clf.fit(x_train,y_train, verbose=100)

0:	learn: 23.7755648	total: 39.4ms	remaining: 2m 17s
100:	learn: 9.2819702	total: 2.99s	remaining: 1m 40s
200:	learn: 3.7870364	total: 5.92s	remaining: 1m 37s
300:	learn: 1.7632262	total: 8.77s	remaining: 1m 33s
400:	learn: 1.0544767	total: 11.6s	remaining: 1m 30s
500:	learn: 0.7963412	total: 14.5s	remaining: 1m 26s
600:	learn: 0.6796257	total: 17.4s	remaining: 1m 24s
700:	learn: 0.6100803	total: 20.7s	remaining: 1m 22s
800:	learn: 0.5596387	total: 24.3s	remaining: 1m 21s
900:	learn: 0.5150573	total: 27.9s	remaining: 1m 20s
1000:	learn: 0.4800518	total: 31.7s	remaining: 1m 19s
1100:	learn: 0.4484667	total: 34.9s	remaining: 1m 16s
1200:	learn: 0.4222831	total: 38s	remaining: 1m 12s
1300:	learn: 0.3971251	total: 41.3s	remaining: 1m 9s
1400:	learn: 0.3752887	total: 45s	remaining: 1m 7s
1500:	learn: 0.3562499	total: 48.4s	remaining: 1m 4s
1600:	learn: 0.3384791	total: 51.9s	remaining: 1m 1s
1700:	learn: 0.3223612	total: 55.4s	remaining: 58.6s
1800:	learn: 0.3083659	total: 59.5s	remaining: 

In [91]:
y_pred=model.predict(x_test)

In [92]:
print( r2_score(y_pred, y_test), mean_squared_error(y_pred, y_test))

0.7677349006367142 5.600896754959119


In [93]:
pred=model.predict(test_x)

In [94]:
submission = pd.DataFrame(data=pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('submission.csv', index=True)

In [95]:
x1_train=np.array(x_train)
y1_train=np.array(y_train)

In [96]:
from keras.layers import Dense, Input, Dropout, Activation, BatchNormalization, Concatenate, Conv1D, Flatten, MaxPooling1D
from keras.models import Model, load_model
from keras import optimizers, callbacks
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras import backend as K
from swa.keras import SWA
from keras import initializers
from keras_radam import RAdam


In [97]:

inps = Input(shape = (39,))

x= Dense(120)(inps)
x= BatchNormalization()(x)
x= Activation('elu')(x)

x= Dense(120)(x)
x= BatchNormalization()(x)
x= Activation('elu')(x)

x= Dense(120)(x)
x= BatchNormalization()(x)
x= Activation('elu')(x)

x= Dense(120)(x)
x= BatchNormalization()(x)
x= Activation('elu')(x)

x= Dense(120)(x)
x= BatchNormalization()(x)
x= Activation('elu')(x)

outs=Dense(1,name='output')(x)

models = Model(inputs=inps, outputs=outs)
models.compile(loss='mse',optimizer=Adam(lr=1.e-5))
models.summary()


Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 39)                0         
_________________________________________________________________
dense_42 (Dense)             (None, 120)               4800      
_________________________________________________________________
batch_normalization_6 (Batch (None, 120)               480       
_________________________________________________________________
activation_6 (Activation)    (None, 120)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 120)               14520     
_________________________________________________________________
batch_normalization_7 (Batch (None, 120)               480       
_________________________________________________________________
activation_7 (Activation)    (None, 120)               0  

In [98]:
x1_train.shape,y1_train.shape

((3801, 39), (3801,))

In [99]:
hist = models.fit(x1_train, y1_train, batch_size= 16, epochs=100, verbose=2, validation_split=0.1)

Train on 3420 samples, validate on 381 samples
Epoch 1/100
 - 2s - loss: 575.1732 - val_loss: 547.7665
Epoch 2/100
 - 1s - loss: 549.6251 - val_loss: 551.9979
Epoch 3/100
 - 1s - loss: 536.7240 - val_loss: 554.8796
Epoch 4/100
 - 1s - loss: 528.9959 - val_loss: 548.9240
Epoch 5/100
 - 1s - loss: 522.5612 - val_loss: 542.6284
Epoch 6/100
 - 1s - loss: 517.1278 - val_loss: 538.6050
Epoch 7/100
 - 1s - loss: 511.8891 - val_loss: 526.0203
Epoch 8/100
 - 1s - loss: 506.0818 - val_loss: 519.7293
Epoch 9/100
 - 1s - loss: 501.5747 - val_loss: 513.1668
Epoch 10/100
 - 1s - loss: 496.9877 - val_loss: 501.6527
Epoch 11/100
 - 1s - loss: 492.4017 - val_loss: 490.1204
Epoch 12/100
 - 1s - loss: 487.8273 - val_loss: 482.9963
Epoch 13/100
 - 1s - loss: 483.3129 - val_loss: 475.3782
Epoch 14/100
 - 1s - loss: 478.5331 - val_loss: 468.9411
Epoch 15/100
 - 1s - loss: 474.0640 - val_loss: 461.9660
Epoch 16/100
 - 1s - loss: 469.8086 - val_loss: 458.0519
Epoch 17/100
 - 1s - loss: 465.7585 - val_loss: 45

KeyboardInterrupt: 